### Script to generate fine tuning data using SIF similarity method

Change the datasets variable according to the requirements

In [ ]:
dataset = "IN" # Options: IN, UK 

In [ ]:
import pandas as pd
import numpy as np
import glob
import json
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')
from utilities import *

In [ ]:
#Reading the documents and summaries 
names, data_source, data_summary = get_summary_data(dataset, "train")
print(len(names))
print(len(data_source))
print(len(data_summary))

In [ ]:
# Loading Model and tokenizer
from transformers import AutoTokenizer
from transformers import  BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = False).to("cuda:1")
bert_model.eval()

In [ ]:
import numpy as np
from flair.data import Sentence
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import csv
from sklearn.metrics.pairwise import cosine_similarity
import torch

#########################################
# HELPER FUNCTIONS
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def load_tsv(f):
    frequencies = {}
    with open(f) as tsv:
        tsv_reader = csv.reader(tsv, delimiter="\t")
        for row in tsv_reader:
            frequencies[row[0]] = int(row[1])

    return frequencies


def load_frequencies(path):
    return load_tsv(path)

with open('freq_UK.json', 'r') as fp:
    freqs = json.load(fp)
total_freq = sum(freqs.values())

#########################################

def get_sen_encoding(sents):
    '''
    Function to generate encoding for each word in the input list 
    input: sents- List of sentences
    return the list of the sentence encoding 
    '''
    a = 0.001
    answer = None
    for sent in sents:
        ip =tokenizer(sent, return_tensors='pt', max_length=100, truncation=True, padding='max_length')
        tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])
#         tokens = [i for i in tokens if i in freqs.keys()]
        ip = ip.to("cuda:1")
        bert_output = bert_model(**ip)
        ends = bert_output['last_hidden_state']
        ends = ends.to("cpu")
        tokens_to_encoding = {}
        for i in range(len(tokens)):
            tokens_to_encoding[tokens[i]] = ends[0][i]
        weights = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens]
        
        with torch.no_grad():
            embedding = np.average([tokens_to_encoding[token] for token in tokens], axis=0, weights=weights)
        if answer == None:
            answer = embedding
            answer.resize_(1, 768)
        else:
            embedding.resize_(1, 768)
            answer = torch.cat((answer, embedding),0)
    return answer
def simi_sif(l1, l2):
    print(len(l1))
    print(len(l2))
    l = l1+l2
    sents_encodings = get_sen_encoding(l)
    embeddings = remove_first_principal_component(np.array(sents_encodings))
    similarities=cosine_similarity(sents_encodings)
    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    print(result)
# simi_sif(split_to_sentences(data_summary[0]), split_to_sentences(data_source[0]))

In [ ]:

def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary 
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary 
    '''
    l = l1+l2
    sents_encodings = get_sen_encoding(l)
    embeddings = remove_first_principal_component(np.array(sents_encodings))
    similarities=cosine_similarity(sents_encodings)
    
    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [ ]:
chunk_summ_word_threshold = 150

def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries 
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries 
    '''
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)
    
    result = (similarity_l_l(summ_sents,doc_sents))
    
    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]
    
    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 1024):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(tokenizer.tokenize(summ)) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ



In [ ]:
#loop to pass every document, generate the fine tuning data and saving in a excel file 
import pandas as pd
training_chunks = []
training_summs = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
#     print(i, len(training_summs), end = ", ", sep = " : ")
    if i%100 == 0: 
        full = list(zip(training_chunks,training_summs))
        df = pd.DataFrame(full,columns=['data', 'summary'])
        df.to_excel("FD_"+dataset+"_SIF.xlsx")
#         break
full = list(zip(training_chunks,training_summs))
df = pd.DataFrame(full,columns=['data', 'summary'])
df.to_excel("FD_"+dataset+"_SIF.xlsx")